#Data Centric Approach
In this notebook we will follow a data centric approach and try to improve predictions for relKa.

In [1]:
#Mount  Google Drive
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My\ Drive/Colab\ Notebooks/ML\ Project2

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/ML Project2


In [18]:
#Usefull Libraries

# pandas / numpy
import pandas as pd
import numpy as np

#import xgboost regressor model
from xgboost import XGBRegressor

#pearson corelation coefficient
from scipy.stats import pearsonr

#import cross validation function from sklearn
from sklearn.model_selection import cross_val_score
#import one hot encoder function from sklearn
from sklearn.preprocessing import OneHotEncoder
#import the train test split function from sklearn to split the dataset randomly for training and testing
from sklearn.model_selection import train_test_split
#import mse loss
from sklearn.metrics import mean_squared_error

In [3]:
#import data from zip
pddata = pd.read_csv('GSM1586785_ScrH-12A_Exd_14mer_cg.csv.zip', compression='zip', error_bad_lines=False)
#print the first samples of the dataframe to get a look at the data
pddata.head(4)

,Unnamed: 0,Kmer,y_1,y_2,y_3,y_4,y_5,y_6,zC_1,zC_2,zC_3,zC_4,zC_5,zC_6,x_1,x_2,x_3,x_4,x_5,x_6,zW_1,zW_2,zW_3,zW_4,zW_5,zW_6,y_7,y_8,y_9,y_10,y_11,y_12,zC_7,zC_8,zC_9,zC_10,zC_11,zC_12,x_7,x_8,...,x_70,x_71,x_72,zW_67,zW_68,zW_69,zW_70,zW_71,zW_72,y_73,y_74,y_75,y_76,y_77,y_78,zC_73,zC_74,zC_75,zC_76,zC_77,zC_78,x_73,x_74,x_75,x_76,x_77,x_78,zW_73,zW_74,zW_75,zW_76,zW_77,zW_78,y_79,y_80,y_81,y_82,y_83,y_84,relKa
0,0,AATGATTAATTACC,0.593697,-1.092779,0.309117,0.133525,0.023403,-0.157348,1.177586,-2.166888,-1.059702,-0.986852,9.851130,-1.990198,-0.398346,0.089837,3.357047,-0.769275,-0.221956,3.386346,0.605164,-4.442468,-2.825918,0.281830,9.129749,-1.237129,0.167000,-1.267301,0.183102,0.141690,0.025378,0.080681,1.958107,-2.755203,-1.608669,-0.641613,9.935603,-1.641385,0.009468,-0.179413,...,-0.219001,-0.545648,3.323891,0.752496,-3.620260,-2.373654,-0.318634,9.522320,-1.373931,0.322805,-0.900553,0.124381,0.036043,0.022354,0.004046,2.020696,-4.257512,-2.142878,0.010588,9.566650,-0.941402,-0.131415,0.153189,3.356950,0.438937,-0.894560,3.707160,2.591673,-3.422281,-1.331358,-0.674709,10.019444,-1.012806,-0.790300,-0.112492,0.118705,0.237596,-0.009713,0.267886,1.00000
1,1,GGTAATTAATCATT,0.790300,-0.112492,0.118705,-0.237596,-0.009713,0.267886,2.591673,-3.422281,-1.331358,-0.674709,10.019444,-1.012806,0.131415,0.153189,3.356950,-0.438937,-0.894560,3.707160,2.020696,-4.257512,-2.142878,0.010588,9.566650,-0.941402,-0.322805,-0.900553,0.124381,-0.036043,0.022354,0.004046,0.752496,-3.620260,-2.373654,-0.318634,9.522320,-1.373931,0.092423,-0.159535,...,-0.117570,-0.826970,3.284579,1.958107,-2.755203,-1.608669,-0.641613,9.935603,-1.641385,-0.167000,-1.267301,0.183102,-0.141690,0.025378,0.080681,0.605164,-4.442468,-2.825918,0.281830,9.129749,-1.237129,0.398346,0.089837,3.357047,0.769275,-0.221956,3.386346,1.177586,-2.166888,-1.059702,-0.986852,9.851130,-1.990198,-0.593697,-1.092779,0.309117,-0.133525,0.023403,-0.157348,1.00000
2,2,AATGATTAATTACT,0.593690,-1.092780,0.309116,0.133525,0.023403,-0.157348,1.177587,-2.166890,-1.059703,-0.986851,9.851129,-1.990198,-0.398346,0.089837,3.357047,-0.769275,-0.221958,3.386346,0.605174,-4.442467,-2.825916,0.281830,9.129752,-1.237126,0.166994,-1.267297,0.183101,0.141690,0.025378,0.080682,1.958105,-2.755201,-1.608675,-0.641611,9.935602,-1.641385,0.009466,-0.179412,...,-0.197861,-0.587698,3.365527,1.077618,-3.502014,-2.362643,-0.283750,9.607518,-1.279651,0.118214,-0.940085,0.118129,0.068205,0.010715,0.111194,1.687236,-4.636734,-2.726127,0.248499,9.282680,-0.900859,0.372109,0.309957,3.365526,0.952859,-0.630216,3.537972,2.137980,-2.777053,-1.371144,-0.839165,10.065669,-1.499679,-0.826872,-1.141872,0.500342,-0.143809,0.043456,0.095141,0.96883
3,3,AGTAATTAATCATT,0.826872,-1.141872,0.500342,0.143809,0.043456,0.095141,2.137980,-2.777053,-1.371144,-0.839165,10.065669,-1.499679,-0.372109,0.309957,3.365526,-0.952859,-0.630216,3.537972,1.687236,-4.636734,-2.726127,0.248499,9.282680,-0.900859,-0.118214,-0.940085,0.118129,-0.068205,0.010715,0.111194,1.077618,-3.502014,-2.362643,-0.283750,9.607518,-1.279651,0.128618,-0.194635,...,-0.117566,-0.826970,3.284579,1.958105,-2.755201,-1.608675,-0.641611,9.935602,-1.641385,-0.166994,-1.267297,0.183101,-0.141690,0.025378,0.080682,0.605174,-4.442467,-2.825916,0.281830,9.129752,-1.237126,0.398346,0.089837,3.357047,0.769275,-0.221958,3.386346,1.177587,-2.166890,-1.059703,-0.986851,9.851129,-1.990198,-0.593690,-1.092780,0.309116,-0.133525,0.023403,-0.157348,0.96883


In [4]:
#convert dataframe to np array
data = pddata.to_numpy()
#labels
relKa = data[:,320:]
# features
# we remove the index of the data samples, since it is not a feature
features = data[:,1:320]
# temporarily remove the 2nd feature because one hot encoding causes ram overflow
features = features[:,1:]

### Preprocessing of data

One hot encoding the data is troubling for my setup and for google colab. The first one is unable to perform the task in reasonable time, while the second one runs out of RAM. I tried both the encoder os scikit learn and a custom encoder of mine that you can find here. The main problem is the size of the end result.

In [5]:
def one_hot_encoder(pos,length):
  """
  One hot encode a list of unique elements

  @param pos: int64 
  @param length: int64 
  """
  encoded = np.zeros(length)
  encoded[pos] = 1
  return encoded

In [6]:
#remove duplicate sequences in case they exist
#Kmer = [one_hot_encoder(i,len(relKa[:,0])) for i in range(len(relKa[:,0]))]
#create the encoding for all sequences
#Kmer_encode = [ for i in range(len(Kmer))]
#Kmer

In [7]:

# initialize a one hot encoder that ignores the unseen sequences
# enc = OneHotEncoder(handle_unknown='ignore')
# use the sequences of the data for the initialization
# enc.fit(Kmer_encode)
# encode the sequences
# enc.transform(Kmer_encode).toarray()


In [8]:
def standardization(x):
  """
  Standardization of elements for a vector x
  @param:x np.ndarray
  """
  mean = np.mean(x)
  #print(mean)
  std = np.std(x)
  #print(std)
  x = np.apply_along_axis(lambda y: (y - mean)/std, 0, x)
  return x

In [9]:
#standardize all features
std_features=np.apply_along_axis(standardization, 1, features)

In [10]:
# randomly split the dataset in a 70/30 split
# change the train_size for partitions of different size
# rerun for different partitions
X_train, X_test, y_train, y_test = train_test_split(features, relKa,  train_size=0.7, random_state=33, shuffle=True)

## Initial Training of the regression method 

We choose Xgboost as our model for this approach.

In [11]:
#Cross validation for initial data and Xgboost
scores = cross_val_score(XGBRegressor(objective='reg:squarederror'), features, relKa, scoring='neg_mean_squared_error')

In [12]:
print("Basic loss for XGBoost:",np.mean((-scores)**0.5))

Basic loss for XGBoost: 0.07645230899873175


In [13]:
#Set the parameters for the XGBRegressor
#we will use the GPU, otherwise it does not train
param_dict = {
    'max_depth':10,
    'n_estimators':1000,
    'objective': 'reg:squarederror',
    'tree_method': 'gpu_hist'
}

In [14]:
#model of XBGRegressor without extensive parameter tuning
model=XGBRegressor(**param_dict)

In [15]:
#fit the model
model.fit(X_train,y_train,eval_metric="rmse")

XGBRegressor(max_depth=10, n_estimators=1000, objective='reg:squarederror',
             tree_method='gpu_hist')

In [20]:
# do some predictions with the model
y_pred = model.predict(X_test)

In [22]:
#mean squared error
mean_squared_error(y_test,y_pred)
#np.corrcoef(y_test,y_pred)

0.00041931443427523427